In [6]:
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain.agents import AgentExecutor, create_react_agent
from langchain import hub
from langchain.memory import ChatMessageHistory
load_dotenv()

True

In [5]:
model = ChatOpenAI(model="gpt-4")
search = TavilySearchResults(max_results=2)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")



In [7]:
memory = ChatMessageHistory(session_id="test-session")

In [9]:
agent = create_react_agent(model, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
from langchain_core.runnables.history import RunnableWithMessageHistory
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [11]:


agent_with_chat_history.invoke(
    {"input": "What is the weather in Nüremberg now?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
I need to use the search tool to find the current weather in Nüremberg.
Action: tavily_search_results_json
Action Input: "Current weather in Nüremberg"[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Nuremberg', 'region': 'Bayern', 'country': 'Germany', 'lat': 49.4478, 'lon': 11.0683, 'tz_id': 'Europe/Berlin', 'localtime_epoch': 1735563467, 'localtime': '2024-12-30 13:57'}, 'current': {'last_updated_epoch': 1735562700, 'last_updated': '2024-12-30 13:45', 'temp_c': 2.6, 'temp_f': 36.7, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 214, 'wind_dir': 'SW', 'pressure_mb': 1032.0, 'pressure_in': 30.48, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 66, 'cloud': 0, 'feelslike_c': 2.1, 'feelslike_f': 35.7, 'windchill_c': 2.1, 'windchill_f': 35.7, 'heatindex_c': 2.6, 'heatindex_f': 36.7, 'dewpoint_c': -3.0, 'de

{'input': 'What is the weather in Nüremberg now?',
 'chat_history': [],
 'output': 'The current weather in Nuremberg is sunny with a temperature of 2.6°C or 36.7°F.'}